In [1]:
import sys
from pathlib import Path

sys.path.append(str(Path.cwd().parent))

In [2]:
from predictor.KAN_predictor import KAN_predictor
from utils.logger import *
import numpy as np
import pandas as pd
from kan import *
from torch import nn, optim

import optuna
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.metrics import roc_auc_score, f1_score, recall_score, precision_score, accuracy_score, r2_score, matthews_corrcoef
from torch.utils.data import DataLoader, TensorDataset, Subset

import os
from pathlib import Path
import shutil
import pickle
from pprint import pprint

In [3]:
latent_dir = Path("/workspace/ToxPred/experimnets/241119_TransformerVAE_encoding/latent")
csv_dir = Path("/workspace/ToxPred/experimnets/241119_TransformerVAE_encoding/data/csv")

with open("/workspace/ToxPred/experimnets/241119_TransformerVAE_encoding/latent/latent_idx_dic.pkl", "rb") as f:
    latent_idx_dic = pickle.load(f)

device = torch.device("cuda" if torch.cuda.is_available else "cpu")
dtype = torch.get_default_dtype()

In [15]:
bace = pd.read_csv(csv_dir / "bace.csv", index_col=0)
bace = bace.loc[latent_idx_dic["bace"]]
print(bace.shape)

x_bace = np.load(latent_dir / "bace_mu.npy")
print(x_bace.shape)
y_bace_c = np.array(bace["Class"])[:,None]
print(y_bace_c.shape)
y_bace_r = np.array(bace["pIC50"])[:,None]
print(y_bace_r.shape)

x_bace_c_tr, x_bace_c_test, y_bace_c_tr, y_bace_c_test = train_test_split(x_bace, y_bace_c, test_size=0.2, random_state=42, stratify=y_bace_c)
x_bace_r_tr, x_bace_r_test, y_bace_r_tr, y_bace_r_test = train_test_split(x_bace, y_bace_r, test_size=0.2, random_state=2024)

print("tr:", x_bace_c_tr.shape, y_bace_c_tr.shape)
print("test:", x_bace_c_test.shape, y_bace_c_test.shape)

print("tr:", x_bace_r_tr.shape, y_bace_r_tr.shape)
print("test:", x_bace_r_test.shape, y_bace_r_test.shape)

(1513, 4)
(1513, 512)
(1513, 1)
(1513, 1)
tr: (1210, 512) (1210, 1)
test: (303, 512) (303, 1)
tr: (1210, 512) (1210, 1)
test: (303, 512) (303, 1)


In [16]:
logger = init_logger(module_name="bace_binary_kan", outdir=Path.cwd()/"bace_binary_kan")

In [17]:
kan_predictor = KAN_predictor(width=[512, 1], device=device, working_dir=Path.cwd(), results_dir_name="bace_binary_kan", LOGGER=logger)

In [18]:
kan_predictor.optimize(trial_num=100, x_tr_val=x_bace_c_tr, y_tr_val=y_bace_c_tr, mode="classification", fold=5, batch_size=128, epochs=150, patience=10)

[20241205-153459] [INFO] |-----Hyperparameters Optimization Start-----|
[20241205-153459] [INFO] -----n_trial: 100  fold: 5  batch_size: 128  epochs: 150  patience: 10-----
[I 2024-12-05 15:34:59,866] A new study created in memory with name: no-name-a7ffeb8c-318c-41a6-9fb1-357db5baba6c
[20241205-153459] [INFO] -----Fold-0-----
[W 2024-12-05 15:35:00,280] Trial 0 failed with parameters: {'lr': 0.0005753773794117186, 'lamb': 0.29334453957883966} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/opt/pip-env/lib/python3.12/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/workspace/ToxPred/MolKAN/src/predictor/KAN_predictor.py", line 66, in obj
    results = self.train(tr_loader, val_loader, **params)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/workspace/ToxPred/MolKAN/src/predictor/KAN_predictor.py", line 84, in train
    self.m

KeyboardInterrupt: 

In [11]:
pprint(kan_predictor.study.best_params, width=1)

{'lamb': 0.0017561038500394375,
 'lr': 0.02121967786728255}


In [10]:
kan_predictor.x_tr_val = x_bace_c_tr
kan_predictor.y_tr_val = y_bace_c_tr
kan_predictor.mode = "classification"
kan_predictor.fold = 5
kan_predictor.batch_size = 128
kan_predictor.epochs = 150
kan_predictor.patience = 20
kan_predictor.kf = StratifiedKFold(kan_predictor.fold, shuffle=True, random_state=42)
kan_predictor.metrics = [roc_auc_score, accuracy_score, precision_score, recall_score, f1_score]
kan_predictor.loss_fn = nn.BCEWithLogitsLoss()

In [11]:
kan_predictor.test(x_bace_c_test, y_bace_c_test, lr=0.02121967786728255, lamb=0.0017561038500394375)

[20241130-121206] [INFO] |-----TEST(emsemble)-----|
[20241130-121206] [INFO] -----Fold-0-----


[20241130-121319] [INFO] EarlyStopping at epoch 23                                                                                            
epoch 23 | tr loss: 1.41e-01 | tr auroc: 9.97e-01 | val loss: 4.90e-01 | val auroc: 8.70e-01:  15%|█▌        | 23/150 [01:10<06:32,  3.09s/it]
[20241130-121319] [INFO] -----Fold-1-----
[20241130-121439] [INFO] EarlyStopping at epoch 26                                                                                            
epoch 26 | tr loss: 1.29e-01 | tr auroc: 9.99e-01 | val loss: 5.71e-01 | val auroc: 8.53e-01:  17%|█▋        | 26/150 [01:19<06:19,  3.06s/it]
[20241130-121440] [INFO] -----Fold-2-----
[20241130-121603] [INFO] EarlyStopping at epoch 27                                                                                            
epoch 27 | tr loss: 9.92e-02 | tr auroc: 9.99e-01 | val loss: 6.76e-01 | val auroc: 8.26e-01:  18%|█▊        | 27/150 [01:22<06:17,  3.07s/it]
[20241130-121604] [INFO] -----Fold-3-----
[20241130-121751

In [5]:
logger = init_logger(module_name="bace_reg_kan", outdir=Path.cwd()/"bace_reg_kan")

In [6]:
kan_predictor = KAN_predictor(width=[512, 1], device=device, working_dir=Path.cwd(), results_dir_name="bace_reg_kan", LOGGER=logger)

In [7]:
kan_predictor.optimize(trial_num=100, x_tr_val=x_bace_r_tr, y_tr_val=y_bace_r_tr, mode="regression", fold=5, batch_size=128, epochs=150, patience=10)

[20241202-181324] [INFO] |-----Hyperparameters Optimization Start-----|
[20241202-181324] [INFO] -----n_trial: 100  fold: 5  batch_size: 128  epochs: 150  patience: 10-----
[I 2024-12-02 18:13:24,160] A new study created in memory with name: no-name-255e5673-79af-4598-869c-26c1a8c47186
[20241202-181324] [INFO] -----Fold-0-----
epoch 149 | tr loss: 1.08e+00 | tr r2: 4.11e-01 | val loss: 1.01e+00 | val r2: 3.64e-01: 100%|██████████| 150/150 [07:16<00:00,  2.91s/it]
[20241202-182042] [INFO] -----Fold-1-----
[20241202-182729] [INFO] EarlyStopping at epoch 138                                                                                       
epoch 138 | tr loss: 1.08e+00 | tr r2: 4.15e-01 | val loss: 1.14e+00 | val r2: 3.28e-01:  92%|█████████▏| 138/150 [06:46<00:35,  2.95s/it]
[20241202-182729] [INFO] -----Fold-2-----
epoch 149 | tr loss: 1.08e+00 | tr r2: 3.69e-01 | val loss: 1.26e+00 | val r2: 4.04e-01: 100%|██████████| 150/150 [07:18<00:00,  2.92s/it]
[20241202-183447] [INFO] -----F

In [8]:
kan_predictor.test(x_test=x_bace_r_test, y_test=y_bace_r_test)

[20241203-173247] [INFO] |-----TEST(emsemble)-----|
[20241203-173247] [INFO] -----Fold-0-----
[20241203-173421] [INFO] EarlyStopping at epoch 31                                                                                      
epoch 31 | tr loss: 1.91e-01 | tr r2: 8.97e-01 | val loss: 5.90e-01 | val r2: 6.28e-01:  21%|██        | 31/150 [01:33<05:59,  3.02s/it]
[20241203-173422] [INFO] -----Fold-1-----
[20241203-173541] [INFO] EarlyStopping at epoch 26                                                                                      
epoch 26 | tr loss: 2.25e-01 | tr r2: 8.76e-01 | val loss: 6.94e-01 | val r2: 5.91e-01:  17%|█▋        | 26/150 [01:18<06:15,  3.03s/it]
[20241203-173542] [INFO] -----Fold-2-----
[20241203-173751] [INFO] EarlyStopping at epoch 43                                                                                      
epoch 43 | tr loss: 1.27e-01 | tr r2: 9.26e-01 | val loss: 6.09e-01 | val r2: 7.12e-01:  29%|██▊       | 43/150 [02:08<05:20,  3.00s/it]


# hERG

In [4]:
herg = pd.read_csv(csv_dir / "herg_karim.csv", index_col=0)
herg = herg.loc[latent_idx_dic["herg_karim"]]
print(herg.shape)

x_herg = np.load(latent_dir / "herg_karim_mu.npy")
print(x_herg.shape)
y_herg = np.array(herg["Y"])[:, None]
print(y_herg.shape)

(13445, 3)
(13445, 512)
(13445, 1)


In [5]:
x_herg_tr, x_herg_test, y_herg_tr, y_herg_test = train_test_split(x_herg, y_herg, test_size=0.2, random_state=42, stratify=y_herg)
print(x_herg_tr.shape, y_herg_tr.shape)
print(x_herg_test.shape, y_herg_test.shape)

(10756, 512) (10756, 1)
(2689, 512) (2689, 1)


In [6]:
logger = init_logger(module_name="herg_kan", outdir=Path.cwd()/"herg_kan")

In [7]:
kan_predictor = KAN_predictor(width=[512, 1], device=device, working_dir=Path.cwd(), results_dir_name="herg_kan", LOGGER=logger)

In [8]:
kan_predictor.optimize(trial_num=100, x_tr_val=x_herg_tr, y_tr_val=y_herg_tr, mode="classification", fold=5, batch_size=512, epochs=150, patience=10)

[20241203-183433] [INFO] |-----Hyperparameters Optimization Start-----|
[20241203-183433] [INFO] -----n_trial: 100  fold: 5  batch_size: 512  epochs: 150  patience: 10-----
[I 2024-12-03 18:34:33,945] A new study created in memory with name: no-name-c0e37ad1-ae41-4b0c-bc2e-0a3695130fd7
[20241203-183433] [INFO] -----Fold-0-----
[20241203-183538] [INFO] EarlyStopping at epoch 10                                                                                            
epoch 10 | tr loss: 6.92e-01 | tr auroc: 5.25e-01 | val loss: 6.93e-01 | val auroc: 5.24e-01:   7%|▋         | 10/150 [01:02<14:39,  6.28s/it]
[20241203-183538] [INFO] -----Fold-1-----
[20241203-183640] [INFO] EarlyStopping at epoch 10                                                                                            
epoch 10 | tr loss: 6.92e-01 | tr auroc: 5.30e-01 | val loss: 6.93e-01 | val auroc: 5.14e-01:   7%|▋         | 10/150 [01:02<14:31,  6.22s/it]
[20241203-183640] [INFO] -----Fold-2-----
[20241203-18374

In [9]:
kan_predictor.test(x_test=x_herg_test, y_test=y_herg_test)

[20241204-182021] [INFO] |-----TEST(emsemble)-----|
[20241204-182021] [INFO] -----Fold-0-----
[20241204-182214] [INFO] EarlyStopping at epoch 19                                                                                            
epoch 19 | tr loss: 4.13e-01 | tr auroc: 9.01e-01 | val loss: 5.43e-01 | val auroc: 8.11e-01:  13%|█▎        | 19/150 [01:53<13:00,  5.95s/it]
[20241204-182215] [INFO] -----Fold-1-----
[20241204-182408] [INFO] EarlyStopping at epoch 19                                                                                            
epoch 19 | tr loss: 4.14e-01 | tr auroc: 9.02e-01 | val loss: 5.63e-01 | val auroc: 8.19e-01:  13%|█▎        | 19/150 [01:52<12:55,  5.92s/it]
[20241204-182409] [INFO] -----Fold-2-----
[20241204-182552] [INFO] EarlyStopping at epoch 17                                                                                            
epoch 17 | tr loss: 4.13e-01 | tr auroc: 9.02e-01 | val loss: 5.81e-01 | val auroc: 8.02e-01:  11%|█▏      

In [10]:
pprint(kan_predictor.study.best_params, width=1)

{'lamb': 0.00034049280398723916,
 'lr': 0.01707324297017968}


In [8]:
kan_predictor.x_tr_val = x_herg_tr
kan_predictor.y_tr_val = y_herg_tr
kan_predictor.mode = "classification"
kan_predictor.fold = 5
kan_predictor.batch_size = 512
kan_predictor.epochs = 150
kan_predictor.patience = 20
kan_predictor.kf = StratifiedKFold(kan_predictor.fold, shuffle=True, random_state=42)
kan_predictor.metrics = [roc_auc_score, accuracy_score, precision_score, recall_score, f1_score]
kan_predictor.loss_fn = nn.BCEWithLogitsLoss()

In [9]:
kan_predictor.test_with_grids_refine(x_herg_test, y_herg_test, lr=0.01707324297017968, lamb=0.00034049280398723916)

[20241205-153720] [INFO] |-----TEST(emsemble) with grids refine-----|
[20241205-153720] [INFO] -----Fold-0-----


[20241205-154011] [INFO] EarlyStopping at epoch 29                                                                                            
epoch 29 | tr loss: 4.01e-01 | tr auroc: 9.13e-01 | val loss: 5.61e-01 | val auroc: 8.09e-01:  19%|█▉        | 29/150 [02:49<11:46,  5.84s/it]
[20241205-154303] [INFO] EarlyStopping at epoch 29                                                                                            
epoch 29 | tr loss: 4.01e-01 | tr auroc: 9.13e-01 | val loss: 5.36e-01 | val auroc: 8.24e-01:  19%|█▉        | 29/150 [02:51<11:56,  5.92s/it]
[20241205-154500] [INFO] EarlyStopping at epoch 19                                                                                            
epoch 19 | tr loss: 4.13e-01 | tr auroc: 9.01e-01 | val loss: 5.20e-01 | val auroc: 8.26e-01:  13%|█▎        | 19/150 [01:55<13:18,  6.09s/it]
[20241205-154501] [INFO] -----Fold-1-----
[20241205-154752] [INFO] EarlyStopping at epoch 29                                                  

# ClinTox

In [4]:
clintox = pd.read_csv(csv_dir / "clintox_M.csv", index_col=0)
clintox = clintox.loc[latent_idx_dic["clintox_M"]]
print(clintox.shape)

x_clintox = np.load(latent_dir / "clintox_M_mu.npy")
print(x_clintox.shape)
y_clintox = np.array(clintox["CT_TOX"])[:, None]
print(y_clintox.shape)

(1468, 3)
(1468, 512)
(1468, 1)


In [ ]:
x_clin_tr, x_clin_test, y_clin_tr, y_clin_test = train_test_split(x_clintox, y_clintox, test_size=0.2, random_state=42, stratify=y_clintox)
print(x_clin_tr.shape, y_clin_tr.shape)
print(x_clin_test.shape, y_clin_test.shape)

(1174, 512) (1174, 1)
(294, 512) (294, 1)


In [6]:
logger = init_logger(module_name="clintox_kan", outdir=Path.cwd()/"clintox_kan")

In [8]:
kan_predictor = KAN_predictor(width=[512, 1], device=device, working_dir=Path.cwd(), results_dir_name="clintox_kan", LOGGER=logger)

In [9]:
kan_predictor.optimize(trial_num=100, x_tr_val=x_clin_tr, y_tr_val=y_clin_tr, mode="classification", fold=5, batch_size=128, epochs=150, patience=20)

[20241206-131446] [INFO] |-----Hyperparameters Optimization Start-----|
[20241206-131446] [INFO] -----n_trial: 100  fold: 5  batch_size: 128  epochs: 150  patience: 20-----
[I 2024-12-06 13:14:46,742] A new study created in memory with name: no-name-d83be2bc-60e9-47d3-8010-d373f812c9b5
[20241206-131446] [INFO] -----Fold-0-----
[20241206-131738] [INFO] EarlyStopping at epoch 54                                                                                            
epoch 54 | tr loss: 2.74e-01 | tr auroc: 5.50e-01 | val loss: 2.71e-01 | val auroc: 4.37e-01:  36%|███▌      | 54/150 [02:50<05:03,  3.16s/it]
[20241206-131738] [INFO] -----Fold-1-----
[20241206-131937] [INFO] EarlyStopping at epoch 37                                                                                            
epoch 37 | tr loss: 2.84e-01 | tr auroc: 5.19e-01 | val loss: 2.67e-01 | val auroc: 6.66e-01:  25%|██▍       | 37/150 [01:58<06:01,  3.20s/it]
[20241206-131937] [INFO] -----Fold-2-----
[20241206-13212

KeyboardInterrupt: 

# CYP3A4

In [4]:
cyp3a4 = pd.read_csv(csv_dir / "cyp3a4_inhib.csv", index_col=0)
cyp3a4 = cyp3a4.loc[latent_idx_dic["cyp3a4_inhib"]]
print(cyp3a4.shape)

x_cyp3a4 = np.load(latent_dir / "cyp3a4_inhib_mu.npy")
print(x_cyp3a4.shape)
y_cyp3a4 = np.array(cyp3a4["Y"])[:, None]
print(y_cyp3a4.shape)

(12319, 3)
(12319, 512)
(12319, 1)


In [5]:
x_cyp3a4_tr_val, x_cyp3a4_test, y_cyp3a4_tr_val, y_cyp3a4_test = train_test_split(x_cyp3a4, y_cyp3a4, test_size=0.2, random_state=42, stratify=y_cyp3a4)
print(x_cyp3a4_tr_val.shape, y_cyp3a4_tr_val.shape)
print(x_cyp3a4_test.shape, y_cyp3a4_test.shape)

(9855, 512) (9855, 1)
(2464, 512) (2464, 1)


In [6]:
logger = init_logger(module_name="cyp3a4_kan")

In [7]:
kan_predictor = KAN_predictor(width=[512, 1], device=device, working_dir=Path.cwd(), results_dir_name="cyp3a4_kan", LOGGER=logger)

In [8]:
kan_predictor.optimize(100, x_cyp3a4_tr_val, y_cyp3a4_tr_val, mode="classification", fold=5, batch_size=512, epochs=150, patience=15, use_amp=True)

[20241207-005910] [INFO] |-----Hyperparameters Optimization Start-----|
[20241207-005910] [INFO] -----n_trial: 100  fold: 5  batch_size: 512  epochs: 150  patience: 15-----
[I 2024-12-07 00:59:10,700] A new study created in memory with name: no-name-98489882-9ae1-43f4-8d13-399c5ef63f05
[20241207-005910] [INFO] -----Fold-0-----
[20241207-010051] [INFO] EarlyStopping at epoch 17                                                                                            
epoch 17 | tr loss: 6.78e-01 | tr auroc: 5.25e-01 | val loss: 6.79e-01 | val auroc: 5.04e-01:  11%|█▏        | 17/150 [01:39<12:57,  5.85s/it]
[20241207-010051] [INFO] -----Fold-1-----
[20241207-010230] [INFO] EarlyStopping at epoch 17                                                                                            
epoch 17 | tr loss: 6.78e-01 | tr auroc: 5.22e-01 | val loss: 6.79e-01 | val auroc: 5.16e-01:  11%|█▏        | 17/150 [01:39<12:54,  5.83s/it]
[20241207-010230] [INFO] -----Fold-2-----
[20241207-01041

KeyboardInterrupt: 

In [9]:
logger = init_logger(module_name="cyp3a4_kan_big_batch")

In [10]:
kan_predictor = KAN_predictor(width=[512, 1], device=device, working_dir=Path.cwd(), results_dir_name="cyp3a4_kan_big_batch", LOGGER=logger)

In [11]:
kan_predictor.optimize(100, x_cyp3a4_tr_val, y_cyp3a4_tr_val, mode="classification", fold=5, batch_size=2048, epochs=150, patience=15, use_amp=True)

[20241207-011907] [INFO] |-----Hyperparameters Optimization Start-----|
[20241207-011907] [INFO] -----n_trial: 100  fold: 5  batch_size: 2048  epochs: 150  patience: 15-----
[I 2024-12-07 01:19:07,545] A new study created in memory with name: no-name-a5792626-7337-40fc-8abe-d4c46e7abf5a
[20241207-011907] [INFO] -----Fold-0-----
[20241207-012000] [INFO] EarlyStopping at epoch 25                                                                                            
epoch 25 | tr loss: 6.77e-01 | tr auroc: 5.35e-01 | val loss: 6.78e-01 | val auroc: 5.31e-01:  17%|█▋        | 25/150 [00:52<04:22,  2.10s/it]
[20241207-012000] [INFO] -----Fold-1-----
[20241207-012058] [INFO] EarlyStopping at epoch 27                                                                                            
epoch 27 | tr loss: 6.77e-01 | tr auroc: 5.38e-01 | val loss: 6.79e-01 | val auroc: 5.08e-01:  18%|█▊        | 27/150 [00:57<04:23,  2.14s/it]
[20241207-012058] [INFO] -----Fold-2-----
[20241207-0121

KeyboardInterrupt: 

In [6]:
logger = init_logger("cyp3a4_kan_big_batch_noamp", outdir="cyp3a4_kan_big_batch")

In [7]:
kan_predictor = KAN_predictor(width=[512, 1], device=device, working_dir=Path.cwd(), results_dir_name="cyp3a4_kan_big_batch", LOGGER=logger)

In [8]:
kan_predictor.optimize(100, x_cyp3a4_tr_val, y_cyp3a4_tr_val, mode="classification", fold=5, batch_size=2048, epochs=300, patience=15, use_amp=False)

[20250110-160156] [INFO] |-----Hyperparameters Optimization Start-----|
[20250110-160156] [INFO] -----n_trial: 100  fold: 5  batch_size: 2048  epochs: 300  patience: 15-----
[I 2025-01-10 16:01:56,684] A new study created in memory with name: no-name-35459545-a901-4d96-80c9-989b656e087f
[20250110-160156] [INFO] -----Fold-0-----
train_start:   0%|          | 0/300 [00:00<?, ?it/s]STAGE:2025-01-10 16:01:58 3491370:3491370 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-01-10 16:02:01 3491370:3491370 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-01-10 16:02:01 3491370:3491370 ActivityProfilerController.cpp:324] Completed Stage: Post Processing
[20250110-160245] [INFO] -------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      

KeyboardInterrupt: 

In [15]:
kan_predictor.test(x_cyp3a4_test, y_cyp3a4_test)

[20241207-174841] [INFO] |-----TEST(emsemble)-----|
[20241207-174841] [INFO] -----Fold-0-----
[20241207-175012] [INFO] EarlyStopping at epoch 43                                                                                            
epoch 43 | tr loss: 3.62e-01 | tr auroc: 9.20e-01 | val loss: 4.67e-01 | val auroc: 8.55e-01:  29%|██▊       | 43/150 [01:30<03:45,  2.11s/it]
[20241207-175013] [INFO] -----Fold-1-----
[20241207-175134] [INFO] EarlyStopping at epoch 38                                                                                            
epoch 38 | tr loss: 3.68e-01 | tr auroc: 9.18e-01 | val loss: 4.70e-01 | val auroc: 8.51e-01:  25%|██▌       | 38/150 [01:20<03:57,  2.12s/it]
[20241207-175135] [INFO] -----Fold-2-----
[20241207-175305] [INFO] EarlyStopping at epoch 43                                                                                            
epoch 43 | tr loss: 3.61e-01 | tr auroc: 9.21e-01 | val loss: 4.71e-01 | val auroc: 8.53e-01:  29%|██▊     

In [16]:
kan_predictor.test_with_grids_refine(x_cyp3a4_test, y_cyp3a4_test)

[20241207-175609] [INFO] |-----TEST(emsemble) with grids refine-----|
[20241207-175609] [INFO] -----Fold-0-----
[20241207-175740] [INFO] EarlyStopping at epoch 43                                                                                            
epoch 43 | tr loss: 3.62e-01 | tr auroc: 9.20e-01 | val loss: 4.67e-01 | val auroc: 8.55e-01:  29%|██▊       | 43/150 [01:30<03:44,  2.10s/it]
[20241207-175740] [INFO] ---grids refine 5 -> 10
[20241207-175813] [INFO] EarlyStopping at epoch 14                                                                                            
epoch 14 | tr loss: 4.23e-01 | tr auroc: 8.87e-01 | val loss: 4.64e-01 | val auroc: 8.56e-01:   9%|▉         | 14/150 [00:32<05:11,  2.29s/it]
[20241207-175813] [INFO] ---grids refine 10 -> 20
[20241207-175845] [INFO] EarlyStopping at epoch 14                                                                                            
epoch 14 | tr loss: 4.23e-01 | tr auroc: 8.87e-01 | val loss: 4.64e-01 | v

In [18]:
pprint(vars(kan_predictor), width=1)

{'LOGGER': <Logger cyp3a4_kan_big_batch_noamp (NOTSET)>,
 '_backward_hooks': OrderedDict(),
 '_backward_pre_hooks': OrderedDict(),
 '_buffers': OrderedDict(),
 '_forward_hooks': OrderedDict(),
 '_forward_hooks_always_called': OrderedDict(),
 '_forward_hooks_with_kwargs': OrderedDict(),
 '_forward_pre_hooks': OrderedDict(),
 '_forward_pre_hooks_with_kwargs': OrderedDict(),
 '_is_full_backward_hook': None,
 '_load_state_dict_post_hooks': OrderedDict(),
 '_load_state_dict_pre_hooks': OrderedDict(),
 '_modules': OrderedDict([('model',
                           MultKAN(
  (act_fun): ModuleList(
    (0): KANLayer(
      (base_fun): SiLU()
    )
  )
  (base_fun): SiLU()
  (symbolic_fun): ModuleList(
    (0): Symbolic_KANLayer()
  )
)),
                          ('loss_fn',
                           BCEWithLogitsLoss()),
                          ('sigmoid',
                           Sigmoid())]),
 '_non_persistent_buffers_set': set(),
 '_parameters': OrderedDict(),
 '_state_dict_hooks': Or

In [20]:
kan_predictor.study.best_params

{'lr': 0.02276235695734735, 'lamb': 0.0005007106505541059}

In [8]:
kan_predictor.x_tr_val = x_cyp3a4_tr_val
kan_predictor.y_tr_val = y_cyp3a4_tr_val
kan_predictor.mode = "classification"
kan_predictor.fold = 5
kan_predictor.batch_size = 2048
kan_predictor.epochs = 150
kan_predictor.patience = 15
kan_predictor.kf = StratifiedKFold(kan_predictor.fold, shuffle=True, random_state=42)
kan_predictor.metrics = [roc_auc_score, accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef]
kan_predictor.loss_fn = nn.BCEWithLogitsLoss()

In [9]:
kan_predictor.test_with_grids_refine(x_cyp3a4_test, y_cyp3a4_test, lr=0.02276235695734735, lamb=0.0005007106505541059)

[20241208-000702] [INFO] |-----TEST(emsemble) with grids refine-----|
[20241208-000702] [INFO] -----Fold-0-----
[20241208-000830] [INFO] EarlyStopping at epoch 43                                                                                            
epoch 43 | tr loss: 3.62e-01 | tr auroc: 9.20e-01 | val loss: 4.67e-01 | val auroc: 8.55e-01:  29%|██▊       | 43/150 [01:26<03:36,  2.02s/it]
[20241208-000830] [INFO] ---grids refine 5 -> 10
[20241208-000902] [INFO] EarlyStopping at epoch 14                                                                                            
epoch 14 | tr loss: 3.99e-01 | tr auroc: 9.05e-01 | val loss: 4.76e-01 | val auroc: 8.47e-01:   9%|▉         | 14/150 [00:30<04:59,  2.20s/it]
[20241208-000902] [INFO] ---grids refine 10 -> 20
[20241208-000934] [INFO] EarlyStopping at epoch 14                                                                                            
epoch 14 | tr loss: 5.85e-01 | tr auroc: 7.57e-01 | val loss: 5.86e-01 | v

RuntimeError: Error(s) in loading state_dict for MultKAN:
	size mismatch for act_fun.0.grid: copying a param with shape torch.Size([512, 12]) from checkpoint, the shape in current model is torch.Size([512, 27]).
	size mismatch for act_fun.0.coef: copying a param with shape torch.Size([512, 1, 8]) from checkpoint, the shape in current model is torch.Size([512, 1, 23]).